Cut out all graphs; and rather just have the coded pieces that differ by section. edge function for each of the following files until new edge method is defined

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import pickle as pkl
import scipy.interpolate
import time
from scipy.signal import savgol_filter
import copy
import statistics 
from scipy.signal import find_peaks
from tabulate import tabulate

In [ ]:
## functions not specific to one method 

def examineinl(edges, binsnum, edgemethodname): #here binsnum is binnumbers 
    idealmids = 0.5 + binsnum    
    adcmids = [sum(i) for i in zip(edges[1:], edges[:-1])] #add together elements
    adcmids[:] = [x / 2 for x in adcmids] #find their mean, verified 3 entires, sufficient
    arraymids = np.array(adcmids)*(-1) 
    inl = [sum(i) for i in zip(idealmids, arraymids)]
    plt.plot(binsnum, inl, color='black', label='inl')
    plt.title(f'{edgemethodname} inl by bin number')
    plt.xlabel("bin number")
    plt.ylabel("inl")
    plt.legend()
    plt.grid()
    return inl 

def makenarrow(bcvar, right, left):
    width = right - left
    width *= (1-bcvar) #reduce the lenght of the bin 
    right = width + left 
    bcvar *= 0.85 #make the change amount reduced by 85% 
    return right, bcvar

def makewide(bcvar, right, left): #function increases the bin width 
    width = right - left
    width *= (1+bcvar)
    right = width + left 
    bcvar *= 0.85 
    return right, bcvar   

## Aug 14

In [ ]:
def readfileterm30k(start): #I wrote this function retroactively to show differences/similarities between methods
    superhist = "histdataset1.pkl"
    with open(superhist, "rb") as f:
        ds = pkl.load(f)
    countarr = ds["C10"] #strictly the compiled im_arr of all the images 
    mbins = np.arange(26500, 26500+len(countarr)) #built into the dataset 
    start = np.argwhere(mbins == start)
    termcountarr = countarr[start[0][0]:] #this starts histogram at 30,000
    termbins = mbins[start[0][0]:] #this is the adc codes which this is valid for
    return termcountarr, termbins 

In [ ]:
termcountarr, termbins = readfileterm30k(30000)
cs = scipy.interpolate.CubicSpline(termbins, termcountarr)

In [ ]:
def buildedges(cs, counts): 
    edges = [30000] #define the first edge to be 30k 
    for a in range(len(counts)): 
        left = edges[-1]
        if left > 103000: #prevent interpolation beyond what we have for values
            break
        inte = cs.integrate(left, left+1) #generically make the width = 1        
        right = left + 1 
        x = 0 #a counter for issues 
        bcvar = 0.25 #bin change variable (start with 25% change)
        while len(edges) != a+2: 
            if counts[a]-0.01 < inte < counts[a] +0.01: #make the requirement sufficient
                edges.append(right)  
            elif counts[a] > inte: #bin is too small, widen right edge
                right, bcvar = makewide(bcvar, right, left)
                inte = cs.integrate(left, right)       
            elif counts[a] < inte:  #bin is too big
                right, bcvar = makenarrow(bcvar, right, left)
                inte = cs.integrate(left, right)
            x +=1 
    return edges

## condensed spline fitting updated 8.24.23/ Condensed spline fitting (1-3) 


Condensed spline fitting is methods 1-3 of this, while the updated is them all. I remove this for redudancy issues

In [ ]:
#fit spline to the filtered outputs 
termcountarr, termbins = readfileterm30k(30000)
smoothed = savgol_filter(termcountarr, 33, 3)
cs = scipy.interpolate.CubicSpline(termbins, smoothed) #spline fitting for the smoothed values
edges = buildedges(cs, termcountarr) 

In [ ]:
## normalize the filtered output 
termcountarr, termbins = readfileterm30k(30000)
smoothed2 = savgol_filter(termcountarr, 33, 3)
correction = sum(termcountarr)/ sum(smoothed2)
print(correction)
smoothed2 *= correction
cs2 = scipy.interpolate.CubicSpline(termbins, smoothed2) #spline fitting for the smoothed values
edges2 = buildedges(cs2, termcountarr)

In [ ]:
#add a factor for normalization to the spline 
smoothed3 = savgol_filter(termcountarr, 33, 3)
cs3 = scipy.interpolate.CubicSpline(termbins, smoothed3)
inte = cs3.integrate(termbins[0], termbins[-1])
corr = sum(termcountarr[:-1])/ inte
print(corr)
## correct the number of counts by dividing by corr factor 
termcountarr2 = termcountarr/corr 
edges3 = buildedges(cs3, termcountarr2)

In [ ]:
## offset the location of the filtered values; normalize the spline
smoothed4 = savgol_filter(termcountarr, 33, 3)
cs4 = scipy.interpolate.CubicSpline(termbins +0.5, smoothed4) #correction suggestion by Steve
inte = cs4.integrate(termbins[0], termbins[-1])
corr = sum(termcountarr[:-1])/ inte
## correct the number of counts by dividing by corr factor 
termcountarr3 = termcountarr/corr 
edges4 = buildedges(cs4, termcountarr3)

In [ ]:
## offset the location of the filtered values; no correction
smoothed5 = savgol_filter(termcountarr, 33, 3)
cs5 = scipy.interpolate.CubicSpline(termbins +0.5, smoothed5) #correction suggestion by Steve
edges5 = buildedges(cs5, termcountarr)

## investigate spline fitting

In [ ]:
## put the cumul values at center 
termcountarr, termbins = readfileterm30k(30000)
smoothed = savgol_filter(termcountarr, 33, 3)
cs1 = scipy.interpolate.CubicSpline(termbins +0.5, smoothed) #spline fitting for the smoothed values
edges1 = buildedges(cs1, termcountarr)

In [ ]:
## normalize the filtered output 
termcountarr, termbins = readfileterm30k(30000)
smoothed2 = savgol_filter(termcountarr, 33, 3)
#add a correction term to account for the mismatch between the outputs
corr = sum(termcountarr)/sum(smoothed2)
smoothed2 *= corr 
cs2 = scipy.interpolate.CubicSpline(termbins +0.5, smoothed2) #spline fitting for the smoothed values
edges2 = buildedges(cs2, termcountarr)

In [ ]:
## fix the interpolation below the starting value and at end 
termcountarr2, termbins2 = readfileterm30k(29999)
smoothed6 = savgol_filter(termcountarr2, 33, 3)
cs6 = scipy.interpolate.CubicSpline(termbins2 +0.5, smoothed6) #spline fitting for the smoothed values
edges6 = buildedges(cs6, termcountarr) #do not make a bin for 29,999 use original hist array 30-103k 

In [ ]:
## fix the interpolation below the starting value and at end; normalize the filtered 
smoothed7 = savgol_filter(termcountarr2, 33, 3)
#add a correction term to account for the mismatch between the outputs
corr = sum(termcountarr2)/sum(smoothed7)
smoothed7 *= corr 
cs7 = scipy.interpolate.CubicSpline(termbins2 +0.5, smoothed7) #spline fitting for the smoothed values
edges7 = buildedges(cs7, termcountarr)

## cumulative sum binning method 

In [ ]:
termcountarr, termbins = readfileterm30k(29999)

In [ ]:
def makeedges(counts, spline):
    edges = [30000]
    for a in range(len(counts)):
        left = edges[-1]
        if left > 102999.5: #prevent interpolation beyond what we have for bins
            break
        Fr = counts[a] + spline(left) #F(r)= cs(right) ideally
        right = left + 1 #standard bin width = 1
        cv = spline(right) #current value, based on current right edge
        bcvar = 0.25 #bin change variable (start with 25% change)  
        while len(edges) != a+2: 
            if Fr-0.01 < cv < Fr +0.01: #specify the requirement
                edges.append(right)
            
            elif cv < Fr: #bin is too small, widen right edge
                right, bcvar = makewide(bcvar, right, left)
                cv = spline(right)
            
            elif cv > Fr:  #bin is too big, reduce right edge
                right, bcvar = makenarrow(bcvar, right, left)
                cv = spline(right)
    return edges

In [ ]:
smoothed = savgol_filter(termcountarr, 33, 3)
cumulsum = np.cumsum(smoothed)
#take the bin values to be at the center of the bin
cs = scipy.interpolate.CubicSpline(termbins +0.5, cumulsum) #spline fitting for the cumulative values 

In [ ]:
edges = makeedges(termcountarr[1:], cs) #exclude 29999 to start the array at 30k

In [ ]:
# determine if bins too large/small 
def makeedgesinvestigate(counts, spline):
    edges = [30000]
    bs = []
    for a in range(len(counts)):
        left = edges[-1]
        if left > 102999.5: #prevent interpolation beyond what we have for bins
            break
        Fr = counts[a] + spline(left) #F(r)= cs(right),

        right = left + 1 #standard bin width = 1
        cv = spline(right) #current value, based on current right edge
        
        bcvar = 0.25 #bin change variable (start with 25% change)  
        
        while len(edges) != a+2: 
            if Fr-0.01 < cv < Fr +0.01: #specify the requirement
                edges.append(right)
                if cv > Fr: #the bin made is wider than necessary 
                    bs.append(-1)
                elif cv < Fr: #the bin made is not wide enough
                    bs.append(1)
            elif cv < Fr: #bin is too small, widen right edge
                right, bcvar = makewide(bcvar, right, left)
                cv = spline(right)
            
            elif cv > Fr:  #bin is too big, reduce right edge
                right, bcvar = makenarrow(bcvar, right, left)
                cv = spline(right)
    return edges, bs


def makeedges4(counts, spline): #change convergence 
    edges = [30000]
    for a in range(len(counts)):
        left = edges[-1]
        if left > 102999.5: #prevent interpolation beyond what we have for bins
            break
        Fr = counts[a] + spline(left) #F(r)= cs(right),

        right = left + 1 #standard bin width = 1
        cv = spline(right) #current value, based on current right edge
        
        bcvar = 0.25 #bin change variable (start with 25% change)  
        
        while len(edges) != a+2: 
            if Fr- 1 < cv < Fr + 1: #specify the requirement
                edges.append(right)
            
            elif cv < Fr: #bin is too small, widen right edge
                right, bcvar = makewide(bcvar, right, left)
                cv = spline(right)
            
            elif cv > Fr:  #bin is too big, reduce right edge
                right, bcvar = makenarrow(bcvar, right, left)
                cv = spline(right)
    return edges

def makeedges2(counts, spline): ##add residual consideration, adding the difference in the next bin 
    start = time.time() #add time
    edges = [30000]
    for a in range(len(counts)):
        left = edges[-1]
        if left > 129999: #prevent interpolation beyond what we have for values
            break      
        
        Fr = counts[a] + spline(left) #this is the goal 
        right = left + 1 #generically start with width = 1 
        cv = spline(right) 

        bcvar = 0.25 #bin change variable (start with 25% change)        
        while len(edges) != a+2: 
            
            if Fr -0.01 < cv < Fr +0.01: #make the requirement sufficient
                edges.append(right)
                
                #account for the over/under estimation in the next bin ?
                if a+1 == len(counts):
                    break
                diff = cv - Fr
                counts[a+1] -= diff
            elif cv < Fr: #bin is too small, widen right edge
                right, bcvar = makewide(bcvar, right, left)
                cv = spline(right)
                
            elif cv > Fr:  #bin is too big, reduce right edge
                right, bcvar = makenarrow(bcvar, right, left)
                cv = spline(right)            
    end = time.time()
    print("execution time", end-start)
    return edges, bs

## Method Comparison

In [ ]:
termcountarr, termbins = readfileterm30k(29999)
smoothed = savgol_filter(termcountarr, 33, 3)
cumulsum = np.cumsum(smoothed)
#take the bin values to be at the center of the bin
cs = scipy.interpolate.CubicSpline(termbins +0.5, cumulsum) #spline fitting for the cumulative values 
edges = makeedges(termcountarr[1:], cs) #exclude 29999 to start the array at 30k

In [ ]:
#(for reference) investigate spline fitting method 2 
#remember this is not complete. you would need to decovolve the psf in order to use this method

csint = scipy.interpolate.CubicSpline(termbins +0.5, smoothed)
intedges = buildedges(csint, termcountarr[1:])

In [ ]:
## ## Spline fitting to countarr; just wrong 
cscount = scipy.interpolate.CubicSpline(termbins[1:], termcountarr[1:])
edgescountarr = buildedges(cscount, termcountarr[1:]) #exclude 29999 to start the array at 30k

## Cumulative binning 9.14.23

uses a different function to read file and doesnt terminate below 30k 

In [ ]:
def checkbins(pcounts, pbins):
    issueindex = []
    for x in range(len(pbins)-1):
        if pbins[x] != pbins[x-1] +1:
            issueindex.append(x)
    if len(issueindex) != 0:
        #take the last index in the list and begin the lists there
        lastindex = issueindex[-1]
        bins = pbins[lastindex:]
        counts = pcounts[lastindex:]
    else: 
        counts = pcounts
        bins = pbins 
    return counts, bins
    
def load_dataset(amp): #load and prepares the data
    superhist = "histdataset1.pkl"
    with open(superhist, "rb") as f:
        ds = pkl.load(f)
    countarr = ds[amp] #strictly the compiled im_arr of all the images 
    mbins = np.arange(26500, 26500+len(countarr)) #built into the pkl file
    mincounts = 150
    scounts = countarr > mincounts #bin indexes of sufficient counts
    pcounts = countarr[scounts]
    pbins = mbins[scounts]
    ##check that bins have no gaps; it will cause issues
    counts, bins = checkbins(pcounts, pbins)
    return counts, bins

def filter_dataset(counts, bins):
    smoothed = savgol_filter(counts, 33, 3) #apply filter
    cumulsum = np.cumsum(smoothed) #make the cumulative sum
    #by construction of the method, cumulative sum values lie on the right edge of the bin (ie bins +1)
    cs = scipy.interpolate.CubicSpline(bins +1, cumulsum) #spline fitting for the cumulative values 
    #i have the values attached to the left edge 
    #print("smoothed sum - counts", cumulsum[-1] - sum(counts))
    return cs 

def examinednl(edges, start):
    widths = []
    for x in range(len(edges)-1):
        width = edges[x+1]- edges[x]
        widths.append(width)
    dnl = np.array(widths) -1  #ideal width =1
    plt.scatter(np.arange(start, start+len(dnl)), dnl, s=5, label='dnl')
    plt.title('dnl by bin')
    plt.xlabel("bin")
    plt.ylabel("dnl")
    plt.grid()
    plt.show()
    return dnl

def checkstartingbin(start, bins): #verifies data exists for starting bin 
    if start < bins[0]:
        print("binning cannot start at specified value")
        print("binning can start at", bins[0]) 
        raise KeyboardInterrupt

def cumulativeedges(counts, bins, spline, un, start):
    st = time.time() #time the algorithm ? 
    checkstartingbin(start, bins)
    edges = [start] #edges start at leftmost bin
    residualcumsum = spline(start) #the cumsum of the bins not included in the binning 
    firstcount = np.argwhere(bins == start) #start using values where specified     
    rightgoals = np.cumsum(counts[firstcount[0][0]:]) #the cumsum of the counts to be achieved; ie the right edge
    goals = rightgoals + residualcumsum #add in the ped of the spline discarded 
    for a in range(len(goals)): 
        left = edges[-1]
        if left > 102999.5: #prevent interpolation beyond what we have for bins
            break
        right = left + 1 #standard bin width = 1
        cv = spline(right) #current value, based on current right edge
        bcvar = 0.25 #bin change variable (start with 25% change)  
        x =0
        while len(edges) != a+2: 
            if goals[a]-un < cv < goals[a] + un: #specify the requirement
                edges.append(right)   
            elif cv < goals[a]: #bin is too small, widen right edge
                right, bcvar = makewide(bcvar, right, left)
                cv = spline(right)
                if x >1500:
                    print("help small", a)
            elif cv > goals[a]:  #bin is too big, reduce right edge
                right, bcvar = makenarrow(bcvar, right, left)
                cv = spline(right)
                if x >1500:
                    print("help big", a)
            x+= 1
    end = time.time()
    print("the computation time is", end-st)
    return edges

In [ ]:
counts, bins = load_dataset("C01")
cs = filter_dataset(counts, bins)
edges = cumulativeedges(counts, bins, cs, 1, 30000)

In [ ]:
def getinl(edges, start): #here binsnum is binnumbers 
    inlbins = np.arange(start, start+len(edges)-1)
    idealmids = 0.5 + inlbins
    adcmids = [sum(i) for i in zip(edges[1:], edges[:-1])] #add together elements
    adcmids[:] = [x / 2 for x in adcmids] #find their mean, verified 3 entires, sufficient
    arraymids = np.array(adcmids)*(-1) 
    inl = [sum(i) for i in zip(idealmids, arraymids)]
    return inl 

def getdnl(edges, start):
    widths = []
    for x in range(len(edges)-1):
        width = edges[x+1]- edges[x]
        widths.append(width)
    dnl = np.array(widths) -1  #ideal width =1
    return dnl

def plotmanyinls(startingvals, counts, bins, cs): 
    dnls = {}
    fig = plt.figure(figsize=(15,12))
    axes = fig.subplots(2,2, sharex=True, sharey=True)
    fig.suptitle('inl by starting bin', fontsize=24)
    plt.setp(axes[1,0], xlabel = 'bin', ylabel = 'inl') #fix the labels so on all of them 
    plt.setp(axes[0,0], ylabel = 'inl')
    a = 0
    b = 0 
    c = 0
    while c < 2: 
        plt.setp(axes[1,c], xlabel = 'bin')
        c += 1 
    for n in range(len(startingvals)): 
        edges = cumulativeedges(counts, bins, cs, 1, startingvals[n])
        inl = getinl(edges, startingvals[n])
        dnl = getdnl(edges, startingvals[n])
        dnls[startingvals[n]] = dnl
        print("Binning beginning at", startingvals[n], "has max and min inl vals", max(inl), min(inl))
        xvals = np.arange(startingvals[n], startingvals[n] + len(inl))
        axes[a,b].plot(xvals, inl)
        axes[a,b].set_title(f"inl by bin starting at {startingvals[n]}")
        axes[a,b].grid()
        if a < 1 : 
            a +=1 
        else: 
            b +=1 
            a = 0
    return dnls 
            
def plotmanydnl(startingvals, dnls):
    fig = plt.figure(figsize=(10,8))
    axes = fig.subplots(2,2, sharex=True, sharey=True)
    fig.suptitle('dnl by starting bin', fontsize=24)
    plt.setp(axes[1,0], xlabel = 'bin', ylabel = 'dnl') #fix the labels so on all of them 
    plt.setp(axes[0,0], ylabel = 'dnl')
    a = 0
    b = 0 
    c = 0
    while c < 2: 
        plt.setp(axes[1,c], xlabel = 'bin')
        c += 1 
    for n in range(len(startingvals)): 
        dnl = dnls[startingvals[n]]
        xvals = np.arange(startingvals[n], startingvals[n] + len(dnl))
        axes[a,b].scatter(xvals, dnl, s=5)
        axes[a,b].set_title(f"dnl by bin starting at {startingvals[n]}")
        axes[a,b].grid()
        if a < 1 : 
            a +=1 
        else: 
            b +=1 
            a = 0

In [ ]:
#use positive method and cut out filtered values that are negative 
def filter_dataset_adjust_new(counts, bins): #can replace the old function with this
    smoothed = savgol_filter(counts, 33, 3) #apply filter
    # require that the result of smoothed > 0 
    positive = smoothed > 0
    positivesmoothed = smoothed[positive]
    positivebins = bins[positive]
    # remove any jumps in the dataset 
    filtered, filtbins = checkbins(positivesmoothed, positivebins) #use same function as before
    #now fit the spline to the cumulative sum of the filtered values greater than zero 
    cumulativesum = np.cumsum(filtered)
    cs = scipy.interpolate.CubicSpline(filtbins +1, cumulativesum) #have right edge
    #remove any counts before where we have positive values from the savgol 
    validcounts = counts[len(bins)-len(filtbins):] 
    return cs, validcounts, filtbins, filtered  

#returns cs, counts, bins (but cut to the proper length)

In [ ]:
counts, bins = load_dataset("C01")
csnew, validcounts, filtbins, filtered = filter_dataset_adjust_new(counts, bins)
newedges = cumulativeedges(validcounts, filtbins, csnew, 1, 30000)

In [ ]:
amplist = ["C00", "C01", "C02", "C03", "C04", "C05", "C06", "C07", 
           "C10", "C11", "C12", "C13", "C14", "C15", "C16", "C17"]

def plotdnls(amplist, start): 
    fig = plt.figure(figsize=(20,18))
    axes = fig.subplots(8,2, sharex=True, sharey=True)
    fig.suptitle('run 13144 dnl vs bin by amp', fontsize=24)
    plt.setp(axes[7,0], xlabel = 'bin', ylabel = 'count') #fix the labels so on all of them 
    plt.setp(axes[7,1], xlabel = 'bin')
    a = 0
    b = 0 
    c = 0 
    while c < 7: 
        plt.setp(axes[c,0], ylabel = 'count')
        c += 1 
    for amp in amplist: 
        counts, bins = load_dataset(amp) #but use the method of throwing out bins that are negative
        cs, ncounts, fbins, filtered = filter_dataset_adjust_new(counts, bins)
        if start == 0: 
            st = fbins[0]
        else: 
            st = start
        edges = cumulativeedges(ncounts, fbins, cs, 1, st) 
        dnl = getdnl(edges, st)
        axes[a,b].scatter(np.arange(st, st+len(dnl)), dnl, s=0.2)
        axes[a,b].set_title(f"{amp}")
        axes[a,b].grid()
        if a < 7: 
            a +=1 
        else: 
            b +=1 
            a = 0

In [ ]:
## find downticks location

fig = plt.figure(figsize=(20,18))
axes = fig.subplots(8,2, sharex=True, sharey=True)
fig.suptitle('run 13144 hist vs bin by amp', fontsize=24)
plt.setp(axes[7,0], xlabel = 'bin', ylabel = 'count') #fix the labels so on all of them 
plt.setp(axes[7,1], xlabel = 'bin')
a = 0
b = 0 
c = 0 
while c < 7: 
    plt.setp(axes[c,0], ylabel = 'count')
    c += 1 

for amp in amplist: 
    counts, bins = load_dataset(amp)
    csnew, validcounts, filtbins, filtered = filter_dataset_adjust_new(counts, bins)
    mins= scipy.signal.find_peaks(-validcounts, distance=80, prominence=[1e4, None], width=2)
    #mins = scipy.signal.find_peaks(-validcounts, distance=32, prominence=[1e3, None], width=2)
    boi = filtbins[mins[0]]
    axes[a,b].plot(filtbins, validcounts, label=f"hist of {amp}")
    axes[a,b].eventplot(boi, color='red', linelengths=5e5, lineoffsets=2.5e5, alpha=0.3, label='find peaks')
    if a < 7: 
        a +=1 
    else: 
        b +=1 
        a = 0

In [ ]:
head = ["amp", "first 10 separations", "mode separation"]
table=[]
for amp in ampsep.keys():
    linetable = [amp, ampsep[amp][0][:10], ampsep[amp][1]]
    table.append(linetable)

print(tabulate(table, headers=head))

In [ ]:
## make a method that recordds the residual for each bin 

def cumulativeedgesresiduals(counts, bins, spline, un, start):
    st = time.time() #time the algorithm ? 
    checkstartingbin(start, bins)
    edges = [start] #edges start at leftmost bin
    residualcumsum = spline(start) #the cumsum of the bins not included in the binning 
    firstcount = np.argwhere(bins == start) #start using values where specified     
    rightgoals = np.cumsum(counts[firstcount[0][0]:]) #the cumsum of the counts to be achieved; ie the right edge
    goals = rightgoals + residualcumsum #add in the ped of the spline discarded 
    
    residuals = []
    for a in range(len(goals)): 
        left = edges[-1]
        if left > 102999.5: #prevent interpolation beyond what we have for bins
            break
        right = left + 1 #standard bin width = 1
        cv = spline(right) #current value, based on current right edge
        bcvar = 0.25 #bin change variable (start with 25% change)  
        x =0
        while len(edges) != a+2: 
            if goals[a]-un < cv < goals[a] + un: #specify the requirement
                edges.append(right)
                resid = goals[a] - cv
                residuals.append(resid)
            elif cv < goals[a]: #bin is too small, widen right edge
                right, bcvar = makewide(bcvar, right, left)
                cv = spline(right)
                if x >1500:
                    print("help small", a)
            elif cv > goals[a]:  #bin is too big, reduce right edge
                right, bcvar = makenarrow(bcvar, right, left)
                cv = spline(right)
                if x >1500:
                    print("help big", a)
            x+= 1
    end = time.time()
    print("the computation time is", end-st)
    return edges, residuals

In [ ]:
newedges, residuals = cumulativeedgesresiduals(validcounts, filtbins, csnew, 10, 30000)